In [136]:
import json
import datetime
import numpy as np
import matplotlib.pyplot as plt
import dateparser
import TwitterSearch as tw
# import emoji as emo

with open('credentials.json', 'rt') as f: 
    credentials = json.load(fp=f)
ts = tw.TwitterSearch(
    consumer_key = credentials['api_key'],
    consumer_secret = credentials['api_secret_key'],
    access_token = credentials['access_token'],
    access_token_secret = credentials['access_token_secret']
)
    
EMOJI_RIBBON = '🎗'
EMOJI_FLAG = '🇪🇸'

In [187]:
def has_emoji(string, emoji):
    return emoji in string


class Tweet(object):

    def __init__(self, raw_data):
        # self.raw_data = raw_data
        self.id = raw_data['id']
        self.text = raw_data['text']
        self.username = raw_data['user']['name']
        self.user_description = raw_data['user']['description']
        self.user_id = raw_data['user']['id']
        # self.date = dateparser.parse(raw_data['created_at'].replace('+0000 ',''))
    
    @property
    def text_has_ribbon(self):
        return has_emoji(self.text, EMOJI_RIBBON)
    
    @property
    def username_has_ribbon(self):
        return has_emoji(self.username, EMOJI_RIBBON)
    
    @property
    def user_description_has_ribbon(self):
        return has_emoji(self.user_description, EMOJI_RIBBON)
    
    @property
    def text_has_flag(self):
        return has_emoji(self.text, EMOJI_FLAG)
    
    @property
    def username_has_flag(self):
        return has_emoji(self.username, EMOJI_FLAG)
    
    @property
    def user_description_has_flag(self):
        return has_emoji(self.user_description, EMOJI_FLAG)
    
    
class TweetCollection(object):
    
    def __init__(self, tweets):
        self.tweets = tweets
    
    def __add__(self, other):
        if isinstance(other, TweetCollection):
            return TweetCollection(self.tweets+other.tweets)
        else:
            raise ValueError(f'{other} is not a TweetCollection')
    
    def __getitem__(self, idx):
        return self.tweets[idx]
    
    def __setitem__(self, idx, tweet):
        if isinstance(tweet, Tweet):
            self.tweets[idx] = tweet
        else:
            raise ValueError(f'{tweet} is not a Tweet')
    
    def append(self, tweet):
        if isinstance(tweet, Tweet):
            self.tweets.append(tweet)
        else:
            raise ValueError(f'{tweet} is not a Tweet')
        
    def filter_by_property(self, property_name):
        return TweetCollection([t for t in self.tweets if t.__getattribute__(property_name)])
    
    @property
    def length(self):
        return len(self.tweets)
    
    @property
    def text_has_ribbon(self):
        return self.filter_by_property('text_has_ribbon')
    
    @property
    def username_has_ribbon(self):
        return self.filter_by_property('username_has_ribbon')
    
    @property
    def user_description_has_ribbon(self):
        return self.filter_by_property('user_description_has_ribbon')
    
    @property
    def text_has_flag(self):
        return self.filter_by_property('text_has_flag')
    
    @property
    def username_has_flag(self):
        return self.filter_by_property('username_has_flag')
    
    @property
    def user_description_has_flag(self):
        return self.filter_by_property('user_description_has_flag')


class DailyOpinionSplit(object):
    
    def __init__(self, date, tweet_collection):
        self.date = date
        self.tweet_collection = tweet_collection
        self.text_has_ribbon = self.tweet_collection.text_has_ribbon.length
        self.username_has_ribbon = self.tweet_collection.username_has_ribbon.length
        self.user_description_has_ribbon = self.tweet_collection.user_description_has_ribbon.length
        self.text_has_flag = self.tweet_collection.text_has_flag.length
        self.username_has_flag = self.tweet_collection.username_has_flag.length
        self.user_description_has_flag = self.tweet_collection.user_description_has_flag.length
        self.neutral = (self.tweet_collection.length - 
                        self.text_has_ribbon - 
                        self.username_has_ribbon -
                        self.user_description_has_ribbon -
                        self.text_has_flag -
                        self.username_has_flag -
                        self.user_description_has_flag)
    
    @property
    def list(self):
        return [
            self.text_has_ribbon,
            self.username_has_ribbon,
            self.user_description_has_ribbon,
            self.neutral,
            self.user_description_has_flag,
            self.username_has_flag,
            self.text_has_flag,
        ]

In [190]:
since = datetime.date(2018,11,29)
until = datetime.date(2018,11,30)
tso = tw.TwitterSearchOrder()
tso.set_keywords(['#Catalunya', '#Cataluña'], or_operator=True)
tso.set_include_entities(False)
tso.set_since(since)
tso.set_until(until)

In [191]:
tweets = TweetCollection([])
for tweet_data in ts.search_tweets_iterable(tso):
    tweets.append(Tweet(tweet_data))

In [194]:
daily_opinion = DailyOpinionSplit(date=since, tweet_collection=tweets)
daily_opinion

{<__main__.DailyOpinionSplit at 0x10adc2320>}

# Stuff

In [166]:
tuo = tw.TwitterUserOrder('Miotroyo2parte') # create a TwitterUserOrder
# start asking Twitter about the timeline
for tweet_data in ts.search_tweets_iterable(tuo):
    flagtweet = Tweet(tweet_data)
    break

In [167]:
flagtweet.username_has_flag

True